In [1]:
import numpy as np
import scipy.optimize as scyopt
import matplotlib.pyplot as plt
import cProfile

import pyaccel, sirius
import mathphys as mp
%load_ext autoreload
%autoreload 1
%aimport lattice_errors

In [3]:
acc      = sirius.SI_V12.create_accelerator()
fam_data = sirius.SI_V12.get_family_data(acc)

In [5]:
um, mrad, percent = 1e-6, 1e-3, 1e-2
config = dict({'mags':dict(),'girder':dict()})

config['mags']['quads'] = dict({'labels':list(),'sigma':dict()})
config['mags']['quads']['labels'] = ['qfa','qda','qdb1','qdb2','qfb','qf1','qf2','qf3','qf4']
config['mags']['quads']['sigma']['x']     = 40 * um * 1
config['mags']['quads']['sigma']['y']     = 40 * um * 1
config['mags']['quads']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['quads']['sigma']['excit'] = 0.05 * percent * 1

config['mags']['sexts'] = dict({'labels':list(),'sigma':dict()})
config['mags']['sexts']['labels'] = ['sfa','sda','sf1j','sd1j','sd2j','sd3j','sf2j',
                                     'sdb','sfb','sf1k','sd1k','sd2k','sd3k','sf2k']
config['mags']['sexts']['sigma']['x']     = 40 * um * 1
config['mags']['sexts']['sigma']['y']     = 40 * um * 1
config['mags']['sexts']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['sexts']['sigma']['excit'] = 0.05 * percent * 1

config['mags']['bc'] = dict({'labels':list(),'sigma':dict()})
config['mags']['bc']['labels'] = ['bc_lf','bc_hf']
config['mags']['bc']['sigma']['x']     = 40 * um * 1
config['mags']['bc']['sigma']['y']     = 40 * um * 1
config['mags']['bc']['sigma']['roll']  = 0.30 * mrad * 1
config['mags']['bc']['sigma']['excit'] = 0.05 * percent * 1
config['mags']['bc']['sigma']['k_dip'] = 0.10 * percent * 1

config['mags']['bendblocks'] = dict({'labels':list(),'sigma':dict()})
config['mags']['bendblocks']['labels'] = ['b1','b2']
config['mags']['bendblocks']['nrsegs'] = [1,1]
config['mags']['bendblocks']['sigma']['x']      = 40 * um * 1
config['mags']['bendblocks']['sigma']['y']      = 40 * um * 1
config['mags']['bendblocks']['sigma']['roll']   = 0.30 * mrad * 1
config['mags']['bendblocks']['sigma']['excit']  = 0.05 * percent * 1
config['mags']['bendblocks']['sigma']['k_dip']  = 0.10 * percent * 1

#config['girder']['x']     = 80 * um * 1
#config['girder']['y']     = 80 * um * 1;
#config['girder']['roll']  = 0.2 * mrad * 1;

errors = lattice_errors.generate_errors(acc,config,fam_data=fam_data,nr_mach=20,cutoff=1,rndtype='gauss',seed=19880419)

In [6]:
machine = lattice_errors.apply_erros(acc,errors,increment=1.0)

    ------------------------------- 
   |   codx [mm]   |   cody [mm]   |
   | (max)   (rms) | (max)   (rms) |
---|-------------------------------|
000|  6.68    2.35 |  6.39    2.38 |
001| 12.36    4.33 |  6.32    2.00 |
002|  5.94    2.16 |  4.30    1.46 |
003|  2.69    0.61 |  6.78    2.57 |
004|  8.71    2.65 |  5.85    2.28 |
005|  5.14    1.39 |  7.14    2.60 |
006|  8.03    2.66 |  3.70    0.93 |
007|  4.53    1.39 |  4.87    1.50 |
008|  5.43    1.83 |  4.02    0.88 |
009|  5.59    1.62 |  4.14    1.46 |
010|  6.33    2.23 |  3.73    1.31 |
011| 12.50    4.33 |  5.07    1.91 |
012| 12.01    4.10 |  3.60    1.21 |
013|  4.39    1.47 |  5.61    1.89 |
014|  4.60    1.25 |  5.47    1.59 |
015|  7.35    2.58 |  2.98    0.93 |
016| 11.41    4.11 |  7.46    2.75 |
017|  8.18    2.44 |  3.76    1.20 |
018|  2.63    0.69 |  6.32    2.30 |
019|  5.83    1.64 |  4.97    1.37 |
------------------------------------


In [8]:
r = lattice_errors.calc_respm_cod(acc,
                                  fam_data['bpm']['index'],
                                  fam_data['ch']['index'],
                                  fam_data['cv']['index'],
                                  symmetry=10)

bba_ind = lattice_errors.get_bba_ind(acc,fam_data['bpm']['index'])

cor_conf = dict({'bpms':fam_data['bpm']['index'],
                 'hcms':fam_data['ch']['index'],
                 'vcms':fam_data['cv']['index'],
                 'sext_ramp':[0,1], 'respm':r, 'bba':bba_ind,
                'bpm_err':dict({'sigma':(30e-6,30e-6),'cutoff':2})})
machine2 = [machine[i][:] for i in range(len(machine))]
gcodx, gcody = lattice_errors.correct_cod(machine2,cor_conf)

correcting closed-orbit distortions
sextupole ramp: [0, 1]
selection of singular values: 280
maximum number of orbit correction iterations:  20
tolerance: 1.00e-05

    -----------------------------------------------------------------------------------------------
   |           codx [um]           |           cody [um]           |  kickx[urad]     kicky[urad]  | (nr_iter|nr_refactor)
   |      all             bpm      |      all             bpm      |                               | [sextupole ramp]
   | (max)   (rms) | (max)   (rms) | (max)   (rms) | (max)   (rms) | (max)   (rms) | (max)   (rms) | 0.00000 1.00000

---|---------------------------------------------------------------|-------------------------------| 


TypeError: invalid index